## 現在の目標(9/4)
### リプはtweetlistから除外する？

### ネガティブなtweet上位とポジティブなtweet上位を表示する!!!

In [59]:
import nega_posi
from natto import MeCab

from requests_oauthlib import OAuth1Session
from collections import defaultdict
import json

CK = 'your CK'                             # Consumer Key
CS = 'your CS'         # Consumer Secret
AT = 'your AT' # Access Token
AS = 'your AS'         # Accesss Token Secert

In [2]:
def get_usid(scname):
    url = "https://api.twitter.com/1.1/users/show.json"
    # とくにパラメータは無い
    params = { "screen_name" : scname}
    # OAuth で GET
    twitter = OAuth1Session(CK, CS, AT, AS)
    req = twitter.get(url, params = params)
    n = 0
    if req.status_code == 200:
        timeline2 = json.loads(req.text)
        print(timeline2["id"])
        return timeline2["id"]
    else:
        return 0

In [50]:
def get_tweet(usid):
    tweet_list=[]
    # タイムライン取得用のURL
    url = "https://api.twitter.com/1.1/statuses/user_timeline.json?count=300"
    # とくにパラメータは無い
    params = { "user_id" : usid}
    # OAuth で GET
    twitter = OAuth1Session(CK, CS, AT, AS)
    req = twitter.get(url, params = params)
    n = 0
    # print(users["id_str"])
    if req.status_code == 200:
        # レスポンスはJSON形式なので parse する
        timeline = json.loads(req.text)
        # 各ツイートの本文を表示
        print(len(timeline))
        for tweet in timeline:
            if '@' not in tweet["text"]:
                tweet_list.append(tweet["text"])
                #print(tweet["text"])
            if len(tweet_list)==100:
                break
        return tweet_list
#         print(tweet_list)        
    else:
        # エラーの場合
        print ("Error: %d" % req.status_code)
        return 0

In [12]:
def calc_score(tweet_list):
    score=0
    sum_score=0
    sc_list=[]
    for idx,tweet in enumerate(tweet_list):
#         print(idx)
        tw1=nega_posi.JudgeClass(tweet)
        tw1.make_wlist()
        score_dict=tw1.make_dict()
        tw1.make_judge_list()
        score=tw1.calc_score(score_dict)
        sum_score+=score
        sc_list.append(score)
    if len(tweet_list) == 0:
        return 0
    else:
        max_idx=search_max_idx(sc_list)
        for i,idx in enumerate(max_idx):
            print("positive "+str(i+1)+"位: "+str(tweet_list[idx]))
        min_idx=search_min_idx(sc_list)
        for i,idx in enumerate(min_idx):
            print("negative "+str(i+1)+"位: "+str(tweet_list[idx]))
        print(max_idx)
        print(min_idx)
        print("全体平均"+str(sum_score/len(tweet_list)))        
        return sum_score/len(tweet_list)

In [5]:
import numpy as np
def search_max_idx(sc_list):
    my_array=np.array(sc_list)
    #探す対象リスト:my_arrayはnumpy
    #例:上位3件
    K = 3
    # ソートはされていない上位k件のインデックス
    unsorted_max_indices = np.argpartition(-my_array, K)[:K]
    # 上位k件の値
    y = my_array[unsorted_max_indices]
    # 大きい順にソートし、インデックスを取得
    max_indices = np.argsort(-y)
    # 類似度上位k件のインデックス
    max_k_indices = unsorted_max_indices[max_indices]
    return max_k_indices

In [6]:
def search_min_idx(sc_list):
    my_array=np.array(sc_list)
    my_array=my_array*(-1)
    #探す対象リスト:my_arrayはnumpy
    #例:上位3件
    K = 3
    # ソートはされていない上位k件のインデックス
    unsorted_max_indices = np.argpartition(-my_array, K)[:K]
    # 上位k件の値
    y = my_array[unsorted_max_indices]
    # 大きい順にソートし、インデックスを取得
    max_indices = np.argsort(-y)
    # 類似度上位k件のインデックス
    max_k_indices = unsorted_max_indices[max_indices]
    return max_k_indices

In [7]:
# def calc_score(tweet_list):
#     score=0
#     sum_score=0
#     max_score=-1
#     min_score=1
#     for idx,tweet in enumerate(tweet_list):
# #         print(idx)
#         tw1=nega_posi.JudgeClass(tweet)
#         tw1.make_wlist()
#         score_dict=tw1.make_dict()
#         tw1.make_judge_list()
#         score=tw1.calc_score(score_dict)
#         if score >max_score:
#             max_idx=idx
#             max_score=score
#         if score < min_score:
#             min_idx=idx
#             min_score=score
#         sum_score+=score
#     if len(tweet_list) == 0:
#         return 0
#     else:
#         print("全体平均"+str(sum_score/len(tweet_list)))
#         #scorelistから上位のindexを取得
#         print("positive:"+ tweet_list[max_idx])
#         print("negative:"+tweet_list[min_idx])
#         return sum_score/len(tweet_list)
# # 

In [8]:
def calc_dev(score,ave,std):
    return (50+10*(score-ave)/std)

In [58]:
##実際の計算!!!
ave=-0.3003982968868934
std=0.06567610108001085
scname=input()
usid=get_usid(scname)
tweet_list=[]
tweet_list=get_tweet(usid)
print(len(tweet_list))
score=calc_score(tweet_list)
dev=calc_dev(score,ave,std)
print("ポジティブ偏差値は : "+str(dev))

KeyboardInterrupt: 

In [ ]:
@ozaken: 62.6739135436439
@nappippi_:60.534409415700125
@yaba_k:50.56213937680392
@12satouT:84.98402371104332